In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
from transformers import RobertaTokenizer, XLMRobertaXLForSequenceClassification
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 6
    grad_step = 4
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 5
    models = ['xlm-roberta-large']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

Training epoch: 1
FINSIHED BATCH: 19 of 1340
FINSIHED BATCH: 39 of 1340
FINSIHED BATCH: 59 of 1340
FINSIHED BATCH: 79 of 1340
FINSIHED BATCH: 99 of 1340
FINSIHED BATCH: 119 of 1340
FINSIHED BATCH: 139 of 1340
FINSIHED BATCH: 159 of 1340
FINSIHED BATCH: 179 of 1340
FINSIHED BATCH: 199 of 1340
FINSIHED BATCH: 219 of 1340
FINSIHED BATCH: 239 of 1340
FINSIHED BATCH: 259 of 1340
FINSIHED BATCH: 279 of 1340
FINSIHED BATCH: 299 of 1340
FINSIHED BATCH: 319 of 1340
FINSIHED BATCH: 339 of 1340
FINSIHED BATCH: 359 of 1340
FINSIHED BATCH: 379 of 1340
FINSIHED BATCH: 399 of 1340
FINSIHED BATCH: 419 of 1340
FINSIHED BATCH: 439 of 1340
FINSIHED BATCH: 459 of 1340
FINSIHED BATCH: 479 of 1340
FINSIHED BATCH: 499 of 1340
FINSIHED BATCH: 519 of 1340
FINSIHED BATCH: 539 of 1340
FINSIHED BATCH: 559 of 1340
FINSIHED BATCH: 579 of 1340
FINSIHED BATCH: 599 of 1340
FINSIHED BATCH: 619 of 1340
FINSIHED BATCH: 639 of 1340
FINSIHED BATCH: 659 of 1340
FINSIHED BATCH: 679 of 1340
FINSIHED BATCH: 699 of 1340
FINSIHE

FINSIHED BATCH: 859 of 1340
FINSIHED BATCH: 879 of 1340
FINSIHED BATCH: 899 of 1340
FINSIHED BATCH: 919 of 1340
FINSIHED BATCH: 939 of 1340
FINSIHED BATCH: 959 of 1340
FINSIHED BATCH: 979 of 1340
FINSIHED BATCH: 999 of 1340
FINSIHED BATCH: 1019 of 1340
FINSIHED BATCH: 1039 of 1340
FINSIHED BATCH: 1059 of 1340
FINSIHED BATCH: 1079 of 1340
FINSIHED BATCH: 1099 of 1340
FINSIHED BATCH: 1119 of 1340
FINSIHED BATCH: 1139 of 1340
FINSIHED BATCH: 1159 of 1340
FINSIHED BATCH: 1179 of 1340
FINSIHED BATCH: 1199 of 1340
FINSIHED BATCH: 1219 of 1340
FINSIHED BATCH: 1239 of 1340
FINSIHED BATCH: 1259 of 1340
FINSIHED BATCH: 1279 of 1340
FINSIHED BATCH: 1299 of 1340
FINSIHED BATCH: 1319 of 1340
FINSIHED BATCH: 1339 of 1340
Validation loss per 100 evaluation steps: 0.43817007541656494
Validation loss per 100 evaluation steps: 0.37885906922142254
Validation loss per 100 evaluation steps: 0.36582476502766287
Validation loss per 100 evaluation steps: 0.3738012882444599
DEV ACC: 0.8407738095238096
DEV Prec

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

Training epoch: 1
FINSIHED BATCH: 19 of 1340
FINSIHED BATCH: 39 of 1340
FINSIHED BATCH: 59 of 1340
FINSIHED BATCH: 79 of 1340
FINSIHED BATCH: 99 of 1340
FINSIHED BATCH: 119 of 1340
FINSIHED BATCH: 139 of 1340
FINSIHED BATCH: 159 of 1340
FINSIHED BATCH: 179 of 1340
FINSIHED BATCH: 199 of 1340
FINSIHED BATCH: 219 of 1340
FINSIHED BATCH: 239 of 1340
FINSIHED BATCH: 259 of 1340
FINSIHED BATCH: 279 of 1340
FINSIHED BATCH: 299 of 1340
FINSIHED BATCH: 319 of 1340
FINSIHED BATCH: 339 of 1340
FINSIHED BATCH: 359 of 1340
FINSIHED BATCH: 379 of 1340
FINSIHED BATCH: 399 of 1340
FINSIHED BATCH: 419 of 1340
FINSIHED BATCH: 439 of 1340
FINSIHED BATCH: 459 of 1340
FINSIHED BATCH: 479 of 1340
FINSIHED BATCH: 499 of 1340
FINSIHED BATCH: 519 of 1340
FINSIHED BATCH: 539 of 1340
FINSIHED BATCH: 559 of 1340
FINSIHED BATCH: 579 of 1340
FINSIHED BATCH: 599 of 1340
FINSIHED BATCH: 619 of 1340
FINSIHED BATCH: 639 of 1340
FINSIHED BATCH: 659 of 1340
FINSIHED BATCH: 679 of 1340
FINSIHED BATCH: 699 of 1340
FINSIHE

...

Training epoch: 1
FINSIHED BATCH: 19 of 1340
FINSIHED BATCH: 39 of 1340
FINSIHED BATCH: 59 of 1340
FINSIHED BATCH: 79 of 1340
FINSIHED BATCH: 99 of 1340
FINSIHED BATCH: 119 of 1340
FINSIHED BATCH: 139 of 1340
FINSIHED BATCH: 159 of 1340
FINSIHED BATCH: 179 of 1340
FINSIHED BATCH: 199 of 1340
FINSIHED BATCH: 219 of 1340
FINSIHED BATCH: 239 of 1340
FINSIHED BATCH: 259 of 1340
FINSIHED BATCH: 279 of 1340
FINSIHED BATCH: 299 of 1340
FINSIHED BATCH: 319 of 1340
FINSIHED BATCH: 339 of 1340
FINSIHED BATCH: 359 of 1340
FINSIHED BATCH: 379 of 1340
FINSIHED BATCH: 399 of 1340
FINSIHED BATCH: 419 of 1340
FINSIHED BATCH: 439 of 1340
FINSIHED BATCH: 459 of 1340
FINSIHED BATCH: 479 of 1340
FINSIHED BATCH: 499 of 1340
FINSIHED BATCH: 519 of 1340
FINSIHED BATCH: 539 of 1340
FINSIHED BATCH: 559 of 1340
FINSIHED BATCH: 579 of 1340
FINSIHED BATCH: 599 of 1340
FINSIHED BATCH: 619 of 1340
FINSIHED BATCH: 639 of 1340
FINSIHED BATCH: 659 of 1340
FINSIHED BATCH: 679 of 1340
FINSIHED BATCH: 699 of 1340
FINSIHE

FINSIHED BATCH: 639 of 1340
FINSIHED BATCH: 659 of 1340
FINSIHED BATCH: 679 of 1340
FINSIHED BATCH: 699 of 1340
FINSIHED BATCH: 819 of 1340
FINSIHED BATCH: 839 of 1340
FINSIHED BATCH: 859 of 1340
FINSIHED BATCH: 879 of 1340
FINSIHED BATCH: 899 of 1340
FINSIHED BATCH: 919 of 1340
FINSIHED BATCH: 939 of 1340
FINSIHED BATCH: 959 of 1340
FINSIHED BATCH: 979 of 1340
FINSIHED BATCH: 999 of 1340
FINSIHED BATCH: 1019 of 1340
FINSIHED BATCH: 1039 of 1340
FINSIHED BATCH: 1059 of 1340
FINSIHED BATCH: 1079 of 1340
FINSIHED BATCH: 1099 of 1340
FINSIHED BATCH: 1119 of 1340
FINSIHED BATCH: 1139 of 1340
FINSIHED BATCH: 1159 of 1340
FINSIHED BATCH: 1179 of 1340
FINSIHED BATCH: 1199 of 1340
FINSIHED BATCH: 1219 of 1340
FINSIHED BATCH: 1239 of 1340
FINSIHED BATCH: 1259 of 1340
FINSIHED BATCH: 1279 of 1340
FINSIHED BATCH: 1299 of 1340
FINSIHED BATCH: 1319 of 1340
FINSIHED BATCH: 1339 of 1340
Validation loss per 100 evaluation steps: 0.18695057928562164
Validation loss per 100 evaluation steps: 0.315134338

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

Training epoch: 1
FINSIHED BATCH: 19 of 1340
FINSIHED BATCH: 39 of 1340
FINSIHED BATCH: 59 of 1340
FINSIHED BATCH: 79 of 1340
FINSIHED BATCH: 99 of 1340
FINSIHED BATCH: 119 of 1340
FINSIHED BATCH: 139 of 1340
FINSIHED BATCH: 159 of 1340
FINSIHED BATCH: 179 of 1340
FINSIHED BATCH: 199 of 1340
FINSIHED BATCH: 219 of 1340
FINSIHED BATCH: 239 of 1340
FINSIHED BATCH: 259 of 1340
FINSIHED BATCH: 279 of 1340
FINSIHED BATCH: 299 of 1340
FINSIHED BATCH: 319 of 1340
FINSIHED BATCH: 339 of 1340
FINSIHED BATCH: 359 of 1340
FINSIHED BATCH: 379 of 1340
FINSIHED BATCH: 399 of 1340
FINSIHED BATCH: 419 of 1340
FINSIHED BATCH: 439 of 1340
FINSIHED BATCH: 459 of 1340
FINSIHED BATCH: 479 of 1340
FINSIHED BATCH: 499 of 1340
FINSIHED BATCH: 519 of 1340
FINSIHED BATCH: 539 of 1340
FINSIHED BATCH: 559 of 1340
FINSIHED BATCH: 579 of 1340
FINSIHED BATCH: 599 of 1340
FINSIHED BATCH: 619 of 1340
FINSIHED BATCH: 639 of 1340
FINSIHED BATCH: 659 of 1340
FINSIHED BATCH: 679 of 1340
FINSIHED BATCH: 699 of 1340
FINSIHE

FINSIHED BATCH: 739 of 1340
FINSIHED BATCH: 759 of 1340
FINSIHED BATCH: 779 of 1340
FINSIHED BATCH: 799 of 1340
FINSIHED BATCH: 819 of 1340
FINSIHED BATCH: 839 of 1340
FINSIHED BATCH: 859 of 1340
FINSIHED BATCH: 879 of 1340
FINSIHED BATCH: 899 of 1340
FINSIHED BATCH: 919 of 1340
FINSIHED BATCH: 939 of 1340
FINSIHED BATCH: 959 of 1340
FINSIHED BATCH: 979 of 1340
FINSIHED BATCH: 999 of 1340
FINSIHED BATCH: 1019 of 1340
FINSIHED BATCH: 1039 of 1340
FINSIHED BATCH: 1059 of 1340
FINSIHED BATCH: 1079 of 1340
FINSIHED BATCH: 1099 of 1340
FINSIHED BATCH: 1119 of 1340
FINSIHED BATCH: 1139 of 1340
FINSIHED BATCH: 1159 of 1340
FINSIHED BATCH: 1179 of 1340
FINSIHED BATCH: 1199 of 1340
FINSIHED BATCH: 1219 of 1340
FINSIHED BATCH: 1239 of 1340
FINSIHED BATCH: 1259 of 1340
FINSIHED BATCH: 1279 of 1340
FINSIHED BATCH: 1299 of 1340
FINSIHED BATCH: 1319 of 1340
FINSIHED BATCH: 1339 of 1340
Validation loss per 100 evaluation steps: 0.6155720353126526
Validation loss per 100 evaluation steps: 0.3702771263

In [6]:
print(best_dev_acc)

0.8407738095238099


In [7]:
print(best_epoch)

1


In [8]:
print(best_dev_precision)

0.5188492063492064


In [9]:
print(best_dev_recall)

0.6175595238095237


In [10]:
print(best_dev_f1score)

0.5442872603586889


In [11]:
print(overall_list_dev_acc)

[[0.811507936507937, 0.8283730158730168, 0.8273809523809529, 0.8407738095238096, 0.830853174603175], [0.825396825396826, 0.8298611111111124, 0.8392857142857147, 0.8402777777777788, 0.8353174603174612], [0.8139880952380956, 0.8298611111111123, 0.8308531746031753, 0.8303571428571437, 0.8382936507936514], [0.8060515873015879, 0.8313492063492065, 0.8273809523809529, 0.835317460317461, 0.828869047619048], [0.8000992063492073, 0.8278769841269845, 0.8333333333333331, 0.8407738095238099, 0.8373015873015881]]


In [12]:
print(overall_list_dev_precision)

[[0.4981646825396826, 0.5106646825396826, 0.5352678571428571, 0.5205853174603172, 0.4327876984126985], [0.5484623015873015, 0.451140873015873, 0.476438492063492, 0.4866071428571428, 0.4568452380952382], [0.5049107142857143, 0.5044642857142857, 0.5328869047619048, 0.4841269841269842, 0.5022321428571429], [0.38095238095238093, 0.48144841269841265, 0.5128472222222223, 0.507936507936508, 0.49429563492063505], [0.4776785714285712, 0.5188492063492064, 0.4957837301587302, 0.4925595238095237, 0.4744543650793651]]


In [13]:
print(overall_list_dev_recall)

[[0.5714285714285714, 0.5657242063492064, 0.5840773809523809, 0.597718253968254, 0.3950892857142857], [0.6294642857142857, 0.4417162698412699, 0.4694940476190476, 0.5396825396825397, 0.4637896825396825], [0.6135912698412698, 0.5613095238095239, 0.5721726190476192, 0.5381944444444444, 0.5373511904761905], [0.3296130952380953, 0.5064484126984127, 0.5897817460317459, 0.5389384920634921, 0.5275297619047619], [0.6175595238095237, 0.6113591269841269, 0.5399801587301587, 0.5250992063492064, 0.48239087301587297]]


In [14]:
print(overall_list_dev_f1score)

[[0.5137991307634161, 0.5158021541950111, 0.5416666666666666, 0.5375850340136056, 0.399248866213152], [0.5692176870748298, 0.43010204081632686, 0.4582199546485263, 0.4935941043083901, 0.4437074829931973], [0.5368055555555555, 0.5112740929705215, 0.5318830309901736, 0.49158163265306115, 0.5037084278155707], [0.3405470521541951, 0.4797548185941042, 0.5294217687074829, 0.5017644557823127, 0.4914824263038548], [0.519791666666667, 0.5442872603586889, 0.5016510770975058, 0.49260912698412695, 0.4605300453514741]]
